In [113]:
import random
import numpy as np

In [114]:
# Modelling Constants
dist_between_stations = [44.8, 16.7, 107.9, 61.2, 14.1, 34.2, 7.2, 24.4, 25.3, 20.7, 5.3, 29.6, 18.2, 10.5, 21.6, 56.6, 4.0, 15.5, 53.2, 17.0, 72.7, 81.0, 18.4, 16.7, 97.3, 104.7, 6.9] # km
vert_dist_to_highway = [467, 500, 200, 200, 600, 330, 200, 875, 530, 640, 450, 200, 300, 1100, 980, 670, 910, 600, 1075, 200, 450, 200, 560, 200, 200, 200, 200, 200] # m

# Look in winter & safety factor on worst case
truck_range = 500
factor_safety = 4
max_allowable_dist = truck_range / factor_safety

# ONRoute proxy
is_onroute = [0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,1,1,1,1] # bool

In [115]:
def genTrial():
    """Generate a random trial
    
    return bool array of length 28
    """
    return random.choices([0, 1], k=28)

In [238]:
# Fitness Function
def fitness(trial):
    """Determine the fitness of a trial
    
    Low Fitness = Bad Solution
    
    Fitness = 0 Means Impossible Solution
    
    return [trial, fitness]
    """
    answer = 0
    alpha1 = 1
    alpha2 = 1
    alpha3 = 1
    alpha4 = 1
    
    # Check if one of the first three stations are chosen
    if sum([trial[0], trial[1], trial[2]]) == 0:
        return [trial, 0]
    
    # Check if one of the last two stations are chosen
    if sum([trial[-1], trial[-2]]) == 0:
        return [trial, 0]
    
    # Check distance of travel between stations is less than max allowable
    travel_distances = []
    temp = 0
    for i,v in enumerate(trial):
        # Case for last element
        if i == len(trial) - 1:
            break
        # if 1 else 0
        if v:
            temp += dist_between_stations[i]
            travel_distances.append(temp)
            temp = 0
        else:
            temp += dist_between_stations[i]
    if max(travel_distances) > max_allowable_dist:
        return [trial, 0]
    
    # Score distance of travel between stations
    mean_travel_distances = sum(travel_distances) / len(travel_distances)
    score_travel_distances = (mean_travel_distances / max_allowable_dist) * 100
    answer += alpha1 * score_travel_distances
    
    # Score vertical distance of travel to station from highway
    vert_distances = sum([v for i,v in zip(trial, vert_dist_to_highway) if i])
    score_vert_distances = (1 - vert_distances / sum(vert_dist_to_highway)) * 100
    answer += alpha2 * score_vert_distances
    
    # Score is OnRoute Station
    onRoute_locations = sum([v for i,v in zip(trial, is_onroute) if i])
    score_onRoute_locations = onRoute_locations / sum(is_onroute) * 100
    answer += alpha3 * score_onRoute_locations
    
    # Check number of stations
    score_number_stations = (1 - sum(trial) / len(trial)) * 100
    answer += alpha4 * score_number_stations
    
    return [trial, answer]

In [270]:
x = genTrial()
y = fitness(x)
while y[1] == 0:
    x = genTrial()
    y = fitness(x)
print(y)

[[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1], 148.8343584214129]
